<a href="https://colab.research.google.com/github/KishenPatel97/Book_Summarization/blob/main/bleu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
from nltk.translate import bleu
nltk.download('punkt')
from nltk import word_tokenize
# from summarize import processBook

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [38]:
hypothesis = "the #### transcript is a written version of each day 's cnn student news program use this transcript to help students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you saw on cnn student news"

reference = "this page includes the show transcript use the transcript to help students with reading comprehension and vocabulary at the bottom of the page, comment for a chance to be mentioned on cnn student news. you must be a teacher or a student age # # or older to request a mention on the cnn student news roll call . the weekly newsquiz tests students' knowledge of even ts in the news"

hy1 = "This text is just for testing purposes"

ref1 = "The article is meant for evaluation"
# Requires text in a tokenized format
score1 = bleu(word_tokenize(reference), word_tokenize(hypothesis))
print(score1)

score2 = bleu([word_tokenize(ref1)], word_tokenize(hyp1))
print(score2)

0.5053483543335474
0.7311104457090247


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [22]:
def processBook(fname):
    """ Function to process .txt books, esp. from Gutenberg.
    PARAMS: fname (str) - filepath to book to be processed into a string
    RETURNS: (str) - the string object containing the text
    """
    book = open(fname)      # open book file
    book_lines = []         # initialize storage
    for line in book.readlines():       # iterate through each line
        book_lines.append(line.strip())     # process each line

    return " ".join(book_lines)         # return processed lines

In [23]:
control1 = processBook('cn_alice.txt')

control2 = processBook('gs_alice.txt')


In [40]:
# Requires text in a tokenized format
score3 = bleu(word_tokenize(control1), word_tokenize(control2))
print(score3)

score3_rev = bleu(word_tokenize(control2), word_tokenize(control1))
print(score3_rev)

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.311900088600225
0.23930988821651522
